In [1]:
import numpy as np
import pandas as pd
from edempy import Deck
from extrapolation import extrapolation
from LaceyClass import LaceyMixingAnalyzer
import matplotlib.pyplot as plt
from rnn.data_loader_rnn import RNNLoader
from rnn.rnn_extrap import plot_particles, import_dict
import matplotlib.pyplot as plt


sim_names = ["Rot_drum_mono", "Rot_drum_binary_mixed", "Rot_drum_400k"]
sim_name = sim_names[-1]
sim_path =rf"V:\GrNN_EDEM-Sims\{sim_name}.dem"

In [2]:
lacey_settings = f"{sim_path[:-4]}_data\Export_Data\Lacey_settings.txt"

with open(lacey_settings, 'r') as file:
    preferences = file.readlines()
    minCoords = np.array([float(i) for i in str(preferences[1]).split(',')])
    maxCoords = np.array([float(i) for i in str(preferences[3]).split(',')])
    bins = np.array([int(i) for i in str(preferences[5]).split(',')])
    cut_off = int(preferences[7])
    plot = str(preferences[9])
    file.close()
    settings = True

lacey = LaceyMixingAnalyzer(minCoords, maxCoords, bins)
rnn = RNNLoader(1,20,sim_path)

In [ ]:
import math

def dynamic_aor():

    delta=np.zeros(len(grid_y))
    index_nonzero=np.zeros(len(grid_x))
    SurfaceZ=np.zeros(shape=(len(grid_y),len(grid_x)))
    SurfaceY=np.zeros(shape=(len(grid_y),len(grid_x)))
    SurfaceX=np.zeros(shape=(len(grid_y),len(grid_x)))
    Coord=np.zeros((1,3))

    for i in range(len(grid_y)):
    #Find surface particles
        for j in range (len(grid_x)):
            index_coord=np.where((Coord[:,0]>(grid_x[j]-bin_size[0]/2)) & (Coord[:,0]<(grid_x[j]+bin_size[0]/2)) & (Coord[:,1]>(grid_y[i]-bin_size[1]/2)) & (Coord[:,1]<(grid_y[i]+bin_size[1]/2)))
            surf=Coord[index_coord]
            #Index zero values and get surface particles
            if surf.shape[0]>0:
                Max=np.argmax(surf[:,2])
                SurfaceX[i][j]=surf[Max,0]
                SurfaceY[i][j]=surf[Max,1]
                SurfaceZ[i][j]=surf[Max,2]
                index_nonzero[j]=j
            else:
                index_nonzero[j]=-1
    #Linear fit to surface particles
    fit=np.polyfit(grid_x[index_nonzero!=-1],SurfaceZ[i][index_nonzero!=-1],1)
    #Calculating angle of repose and statistics
    delta[i]=math.atan(abs(fit[0]))*180/math.pi    

In [8]:
bins[-1] = 1
delta_t = 0.05
extrapolated_lacey = []
extrapolated_time = []
id_dict_path = rf"V:\GrNN_EDEM-Sims\{sim_name}_data\Export_Data"
id_dict = import_dict(id_dict_path, "id_dict")
plots_path = rf"{sim_path[:-4]}_data\Export_Data\extrap_algo_plots"

# b_coords_lacey, div_size_lacey = lacey.grid()
# for timestep in np.arange(rnn.start_t, rnn.end_t+delta_t, delta_t):
#     timestep_i = rnn.find_nearest(timestep, rnn.deck.timestepValues)
#     particles = rnn.get_particle_data(timestep_i)
#     particles = np.delete(particles, [3,4,5], 1)

#     plot_filename = rf"{plots_path}\{timestep:.2f}.png"
#     pred_t_mass = plot_particles(particles, id_dict, True, timestep, plot_path=plot_filename)
#     mass_1, mass_2, conc = lacey.bining(b_coords_lacey, div_size_lacey, pred_t_mass, cut_off)
#     Lacey_index = lacey.Lacey(mass_1, mass_2, conc, cut_off, len(pred_t_mass))
#     print(f"{timestep}: {Lacey_index}")
#     extrapolated_lacey.append(Lacey_index)
#     extrapolated_time.append(timestep)

# np.savetxt(rf"{plots_path}\_lacey.csv", np.column_stack((extrapolated_time, extrapolated_lacey)), delimiter=",")